In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import tensorflow as tf

# Import and read the charity_data.csv
url = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
application_df = pd.read_csv(url)

# Create a new DataFrame with only the columns needed for modeling
model_df = application_df.drop(columns=["EIN", "NAME"])

# Determine the number of unique values in each column
unique_value_counts = model_df.nunique()

# Binning for APPLICATION_TYPE column
counts = model_df["APPLICATION_TYPE"].value_counts()
application_types_to_replace = list(counts[counts < 500].index)
model_df["APPLICATION_TYPE"] = model_df["APPLICATION_TYPE"].replace(application_types_to_replace, "Other")

# Binning for CLASSIFICATION column
counts_binning = model_df["CLASSIFICATION"].value_counts()
counts_classification = counts_binning[counts_binning > 1]
classifications_to_replace = list(counts_binning[counts_binning < 100].index)
model_df["CLASSIFICATION"] = model_df["CLASSIFICATION"].replace(classifications_to_replace, "Other")

# Convert categorical data to numeric with `pd.get_dummies`
model_df = pd.get_dummies(model_df, dtype=np.float32, sparse=True)

# Split our preprocessed data into our features and target arrays
y = model_df["IS_SUCCESSFUL"].values
X = model_df.drop("IS_SUCCESSFUL", axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 256
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 64
hidden_nodes_layer4 = 32
nn = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"),
    tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"),
    tf.keras.layers.Dense(units=1, activation="sigmoid")
])

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])

# Define the EarlyStopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, validation_split=0.15, epochs=100, batch_size=64, callbacks=[early_stopping])





Epoch 1/100
342/342 [==============================] - 3s 5ms/step - loss: 0.5803 - accuracy: 0.7166 - val_loss: 0.5485 - val_accuracy: 0.7393
Epoch 2/100
342/342 [==============================] - 2s 4ms/step - loss: 0.5613 - accuracy: 0.7264 - val_loss: 0.5462 - val_accuracy: 0.7354
Epoch 3/100
342/342 [==============================] - 2s 5ms/step - loss: 0.5549 - accuracy: 0.7287 - val_loss: 0.5475 - val_accuracy: 0.7339
Epoch 4/100
342/342 [==============================] - 2s 6ms/step - loss: 0.5542 - accuracy: 0.7290 - val_loss: 0.5450 - val_accuracy: 0.7359
Epoch 5/100
342/342 [==============================] - 2s 5ms/step - loss: 0.5530 - accuracy: 0.7314 - val_loss: 0.5480 - val_accuracy: 0.7359
Epoch 6/100
342/342 [==============================] - 2s 5ms/step - loss: 0.5521 - accuracy: 0.7317 - val_loss: 0.5441 - val_accuracy: 0.7341
Epoch 7/100
342/342 [==============================] - 2s 4ms/step - loss: 0.5499 - accuracy: 0.7303 - val_loss: 0.5429 - val_accuracy: 0.7362

In [4]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5540 - accuracy: 0.7290 - 698ms/epoch - 3ms/step
Loss: 0.5539595484733582, Accuracy: 0.7289795875549316


In [5]:
print(f"Accuracy: {model_accuracy:.2f}")

Accuracy: 0.73


In [ ]:
# Increased the number of nodes in each hidden layer, adds a fourth hidden layer, 
# and changes the optimizer to Adam with a learning rate of 0.001. 
# Added a fourth hidden layer and increases the dropout rate to 0.2 to prevent overfitting. 
# Finally, it uses the same early stopping callback as before to stop the training if the validation loss does not improve 
# for 10 epochs. These changes should help improve the performance of the model.

In [ ]:
# This model has 4 hidden layers with 256, 128, 64 and 32 nodes, respectively. 
# Added dropout layers to prevent overfitting